## TODO

#### coding
- Adopt standard variable convention for op, op_str, etc.
    - matrixoperators are given name mat_op, singletrace are given st_op
    - op string names 'X', 'P' are op_str
    - operator tuples are op

- how to handle 'empty' operator SingleTraceOperator(data={():0})
- related, but maybe distinct: how to handle 'zero' operator like A - A = 0?

#### constraints
- inspect constraints, make sure they look good
- I'm confused about the reality constraint, and the fact that the expectation values are either real or imaginary.
- For the models studied in Han et al, expectation values of odd-degree O's vanish. This does not seem to be imposed by the linear constraints. Does it come from the quadratic ones? Or is it a discrete symmetry that needs to be separately imposed?

### efficient implementation
- can I reduce matrix sizes by discarding redundant constraints?
- can I make my bootstrap matrix block diagonal?
- use sparse representations as early as possible

In [1]:
from typing import Union, Self
from numbers import Number
from itertools import chain, product

import numpy as np
import sympy as sp
import cvxpy as cp

import scipy
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
from scipy.linalg import qr
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import splu, svds
from sksparse.cholmod import cholesky

from bmn.algebra import MatrixOperator, SingleTraceOperator, MatrixSystem
from bmn.linear_algebra import get_null_space, create_sparse_matrix_from_dict
from bmn.bootstrap import BootstrapSystem
from bmn.solver import minimal_eigval, sdp_init, sdp_relax, sdp_minimize, minimize, quad_constraint

## One Matrix Model

In [2]:
matrix_system = MatrixSystem(
    operator_basis=['X', 'Pi'],
    commutation_rules_concise = {
        #('P', 'X'): -1j,
        ('Pi', 'X'): 1, # use Pi' = i P to ensure reality
    }
)

# scale variables as P = sqrt(N) P', X = sqrt(N) X'
hamiltonian = SingleTraceOperator(
        #data={("P", "P"): 1, ("X", "X"): 1, ("X", "X", "X", "X"): 7}
        data={("Pi", "Pi"): -1, ("X", "X"): 1, ("X", "X", "X", "X"): 7}
    )

# <tr G O > = 0 might need to be applied only for O with deg <= L-2
#gauge = MatrixOperator(data={('X', 'P'): 1j, ('P', 'X'): -1j, ():1})
gauge = MatrixOperator(data={('X', 'Pi'): 1, ('Pi', 'X'): -1, ():1})

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge=gauge,
    half_max_degree=2
)
#bootstrap.operator_list[:bootstrap.psd_matrix_dim]
bootstrap.matrix_system.commutation_rules

Assuming all operators are hermitian!


{('X', 'X'): 0, ('Pi', 'X'): 1, ('X', 'Pi'): -1, ('Pi', 'Pi'): 0}

In [3]:
# get the quantities needed for numerics
bootstrap_array_sparse, quadratic_constraints = bootstrap.build_optimization_problem()

In [4]:
# testing
op = SingleTraceOperator(data={(): 0, ('Pi', 'Pi', 'Pi'): 1, ('X', 'Pi', 'X', 'Pi'): 2})

param_vector = bootstrap.single_trace_to_coefficient_vector(op)
param_vector_null = bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True)
param_vector.shape, param_vector_null.shape

((31,), (17,))

## check optimize.py code

In [5]:
minimal_eigval(bootstrap_array_sparse, param_vector_null)

-1.0101525445522104

In [6]:
quad_constraint(quadratic_constraints, param=param_vector_null, compute_grad=True)

(array([-3.67346939e-01,  3.67346939e-01, -8.16326531e-02,  5.64539105e-17,
        -8.16326531e-02,  8.16326531e-02,  8.16326531e-02, -8.16326531e-02,
        -3.20169868e-17, -1.02040816e-02,  1.70034384e-16,  8.97959184e-01,
        -9.18367347e-02, -8.16326531e-02,  8.16326531e-02,  9.48979592e-01,
        -8.97959184e-01,  1.98025646e-16, -8.46938776e-01,  8.16326531e-02]),
 array([[-2.27179395e-16,  1.74209492e-01,  7.60318552e-01,
          1.02569894e-02,  0.00000000e+00, -4.62520449e-03,
         -2.24489796e-01,  2.24489796e-01, -1.96054150e-17,
          7.75200746e-01,  2.24489796e-01, -2.24489796e-01,
          1.96054150e-17,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 2.27179395e-16, -1.74209492e-01, -7.60318552e-01,
         -1.02569894e-02,  0.00000000e+00,  4.62520449e-03,
          2.24489796e-01, -2.24489796e-01,  1.96054150e-17,
         -7.75200746e-01, -2.24489796e-01,  2.24489796e-01,
         -1.96054150e-17,  0.00000

In [7]:
# get the A, b objects
num_variables = bootstrap.param_dim_null
op_cons = [SingleTraceOperator(data={(): 1})]

A_op = sparse.csr_matrix((0, num_variables))
b_op = np.zeros(0)
for op in op_cons:
    A_op = sparse.vstack([A_op, sparse.csr_matrix(bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True))])
    b_op = np.append(b_op, 1)

In [8]:
sdp_init(
    bootstrap_array_sparse=bootstrap_array_sparse,
    A=A_op,
    b=b_op,
    init=np.zeros(bootstrap.param_dim_null)
    )

                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) May 26 10:04:31 AM: Your problem has 17 variables, 49 constraints, and 0 parameters.
(CVXPY) May 26 10:04:31 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 26 10:04:31 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 26 10:04:31 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 26 10:04:31 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 26 10:04:31 AM: Compiling problem (target solver=SCS).
(CVX

array([-6.73883565e-11, -7.05476458e-02, -3.07897597e-01, -4.15365685e-03,
       -4.65878746e-13,  1.87301668e-03,  9.09090914e-02, -9.09090914e-02,
       -4.15768214e-11, -3.13924270e-01, -9.09090914e-02,  9.09090914e-02,
        3.82834273e-11, -4.65878746e-13, -1.18197816e-11,  8.52638743e-12,
        2.93218580e-11])

In [9]:
sdp_relax(
    bootstrap_array_sparse=bootstrap_array_sparse,
    A=A_op,
    b=b_op,
    init=np.zeros(bootstrap.param_dim_null),
    radius = 1.0
)

                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) May 26 10:04:31 AM: Your problem has 17 variables, 49 constraints, and 0 parameters.
(CVXPY) May 26 10:04:31 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 26 10:04:31 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 26 10:04:31 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 26 10:04:31 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 26 10:04:31 AM: Compiling problem (target solver=SCS).
(CVX

array([ 3.39265245e-11, -1.94007139e-01, -8.60266989e-01, -2.49661687e-02,
       -9.73735825e-12,  1.86943121e-02,  2.50001650e-01, -2.50001325e-01,
       -1.85906688e-09, -8.49753321e-01, -2.50001325e-01,  2.50001650e-01,
       -1.86682310e-09, -9.73735825e-12, -3.44490535e-09, -3.44022235e-09,
        1.98525576e-07])

In [10]:
sdp_minimize(
    vec = np.abs(np.random.normal(size=bootstrap.param_dim_null)),
    bootstrap_array_sparse=bootstrap_array_sparse,
    A=A_op,
    b=b_op,
    init=np.zeros(bootstrap.param_dim_null),
    radius = 1.0
)

                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) May 26 10:04:32 AM: Your problem has 17 variables, 50 constraints, and 0 parameters.
(CVXPY) May 26 10:04:32 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 26 10:04:32 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 26 10:04:32 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 26 10:04:32 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 26 10:04:32 AM: Compiling problem (target solver=SCS).
(CVX

/Users/gavinhartnett/miniconda3/envs/bmnsim/lib/python3.12/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [12]:
# be careful about argument passing, pass in minimal things
# consider putting all the SDP shit in a solver class

minimize(
    bootstrap=bootstrap,
    op=hamiltonian,
    bootstrap_array_sparse=bootstrap_array_sparse,
    quad_cons=quadratic_constraints,
    op_cons=op_cons,
    init=np.zeros(bootstrap.param_dim_null),
    maxiters=100,
    )

Starting from scratch...
                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) May 26 10:08:29 AM: Your problem has 17 variables, 49 constraints, and 0 parameters.
(CVXPY) May 26 10:08:29 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 26 10:08:29 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 26 10:08:29 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 26 10:08:29 AM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 26 10:08:29 AM: Compiling problem 

array([-1.15467811e-05, -3.67527890e-02, -1.58576026e-01,  6.42491336e-03,
        2.60140006e-02, -9.08516501e-03,  1.05395727e-01, -1.75326363e-03,
        2.56947765e-01, -1.82718849e-01,  8.80387246e-03,  9.94792901e-02,
        2.61514168e-01, -2.35516082e-02,  2.58844297e-01,  2.19545971e-01,
        1.39210295e+00])

In [ ]:
op = SingleTraceOperator(data={(): 0, ('Pi', 'Pi', 'Pi'): 1, ('X', 'Pi', 'X', 'Pi'): 0})

param_vector = bootstrap.single_trace_to_coefficient_vector(op)
param_vector_null = bootstrap.single_trace_to_coefficient_vector(op, return_null_basis=True)

In [ ]:
param_vector

In [ ]:
param_vector_null

In [ ]:
bootstrap.null_space_matrix @ bootstrap.null_space_matrix.T

In [ ]:
op_cons = SingleTraceOperator(data={('X'): 1})
bootstrap.single_trace_to_coefficient_vector(op_cons)

In [ ]:
bootstrap.single_trace_to_coefficient_vector()

In [ ]:
OP1 = SingleTraceOperator(
    data={('X1', 'X2'): 1, ('P1', 'X2'): 7}
    )
OP2 = SingleTraceOperator(
    data={('P1',): 1, ('P2',): 5 * 1j, ('X1', 'X1', 'P2',): -3 * 1j}
    )
print(OP1)
print(OP2)

In [ ]:
bootstrap.operator_list[0], bootstrap.generate_hamiltonian_constraints()[0]

In [ ]:
5_000_00

In [ ]:
bootstrap.operator_list[1], bootstrap.generate_hamiltonian_constraints()[1]

In [ ]:
bootstrap.operator_list[2], bootstrap.generate_hamiltonian_constraints()[2]

In [ ]:
bootstrap.operator_list[3], 1j * 1/2 * bootstrap.generate_hamiltonian_constraints()[3]

In [ ]:
bootstrap.generate_hamiltonian_constraints()[3], bootstrap.generate_hamiltonian_constraints()[3].hermitian_conjugate()

In [ ]:
bootstrap.operator_list[0], -1j * bootstrap.generate_gauge_constraints()[0]

In [ ]:
# this reproduces Eq 14 in the main text (promote this to a unit test)
idx = bootstrap.operator_list.index(('X', 'P', 'P', 'P'),)
bootstrap.generate_cyclic_constraints()[idx]

## OLD

In [ ]:
nu = 1.0

matrix_system = MatrixSystem(
    operator_basis=['X0', 'X1', 'X2', 'P0', 'P1', 'P2'],
    commutation_rules_concise = {
        ('P0', 'X0'): -1j,
        ('P1', 'X1'): -1j,
        ('P2', 'X2'): -1j,
    }
)

# scale variables as P = sqrt(N) P', X = sqrt(N) X'
hamiltonian = SingleTraceOperator(
        data={
           # kinetic term
            ("P0", "P1"): 1,
            ("P1", "P1"): 1,
            ("P1", "P1"): 1,
            # quadratic term
            ('X0', 'X0'): nu**2 / 2,
            ('X1', 'X1'): nu**2 / 2,
            ('X2', 'X2'): nu**2 / 2,
            # cubic term
            ('X0', 'X1', 'X2'): 6 * 1j * nu,
            # quadratic term (XY)
            ('X0', 'X1', 'X0', 'X1'): - 1/4,
            ('X1', 'X0', 'X1', 'X0'): -1/4,
            ('X0', 'X1', 'X1', 'X0'): 1/4,
            ('X1', 'X0', 'X0', 'X1'): 1/4,
            # quadratic term (XZ) TODO check sign
            ('X0', 'X2', 'X0', 'X2'): - 1/4,
            ('X2', 'X0', 'X2', 'X0'): -1/4,
            ('X0', 'X2', 'X2', 'X0'): 1/4,
            ('X2', 'X0', 'X0', 'X2'): 1/4,
            # quadratic term (YZ)
            ('X1', 'X2', 'X1', 'X2'): - 1/4,
            ('X2', 'X1', 'X2', 'X1'): -1/4,
            ('X1', 'X2', 'X2', 'X1'): 1/4,
            ('X2', 'X1', 'X1', 'X2'): 1/4,
            }
            )

# <tr G O > = 0 might need to be applied only for O with deg <= L-2
gauge = MatrixOperator(
    data={
        ('X0', 'P0'): 1j,
        ('P0', 'X0'): -1j,
        ('X1', 'P1'): 1j,
        ('P1', 'X1'): -1j,
        ('X2', 'P2'): 1j,
        ('P2', 'X2'): -1j,
        ():1}
    )

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge=gauge,
    half_max_degree=2
)
#bootstrap.operator_list[:bootstrap.psd_matrix_dim]
bootstrap.matrix_system.commutation_rules

In [ ]:
bootstrap.build_linear_constraints()